In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


import ising


import numpy as np
import matplotlib.pyplot as plt
import os
import re

### Load and prepare data

In [2]:
foldername = "100x100_random"
extension = "bin"

all_files = np.array([f for f in os.listdir(foldername) if f.endswith(extension)])
Ts = np.zeros(len(all_files))
Hs = np.zeros(len(all_files))

for i,name in enumerate(all_files):
    Ts[i],Hs[i] = re.findall(r'-?\d*\.?\d+', name)

arg_T = np.argsort(Ts)
all_files = all_files[arg_T]
Ts = Ts[arg_T]
Hs = Hs[arg_T]


In [ ]:

indices_low_temp = [0]
indices_high_temp = [-1,-2]

 
grids_l,L,n_configs,T,H = ising.open_configurations(foldername + "/" + all_files[indices_low_temp[0]])
grids_h,L,n_configs,T,H = ising.open_configurations(foldername + "/" + all_files[indices_high_temp[0]])

data_low_temp = np.reshape(grids_l,(n_configs,1,L,L))
data_high_temp  = np.reshape(grids_h,(n_configs,1,L,L))

for i in indices_low_temp[1:]:
    grids,L,n_configs,T,H = ising.open_configurations(foldername + "/" + all_files[i])
    data_low_temp = np.vstack((data_low_temp,np.reshape(grids,(n_configs,1,L,L))))

for i in indices_high_temp[1:]:
    grids,L,n_configs,T,H = ising.open_configurations(foldername + "/" + all_files[i])
    data_high_temp = np.vstack((data_high_temp,np.reshape(grids,(n_configs,1,L,L))))

X = np.vstack( (data_low_temp,data_high_temp) )
Y = np.hstack( (np.ones(len(data_low_temp)),np.zeros(len(data_high_temp))) )




In [9]:
Xt = torch.from_numpy(X).float()
Yt = torch.from_numpy(Y).float()

dataset = TensorDataset(Xt, Yt)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:

class IsingFullyCNN(nn.Module):
    def __init__(self):
        super().__init__()


        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
        )
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(32, 2)



    def forward(self, x):

        x = F.conv(x)
        x = self.gap(x) 
        x = x.view(x.size(0), -1)

        return self.fc(x)        # logits


model = IsingFullyCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train_step(x, y):
    optimizer.zero_grad()
    logits = model(x)
    loss = criterion(logits, y)
    loss.backward()
    optimizer.step()
    return loss.item()